In [1]:
#imports

#exec(open("0a Seminar Imports.ipynb").read())
%run ./"0a Seminar Imports.ipynb"


/home/ben/miniconda3/bin/python
3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
astropy 5.1.dev153+gb740594dc
reproject 0.8
spectral_cube 0.6.1.dev22+g003ef16
/home/ben/.local/lib/python3.8/site-packages/astrodendro/__init__.py


# Part 2: Reprojection

In [2]:

#Just Using CO 3-2 right now
files = ['HCN_J1-0.cube.fits']
#files=["Pre-Smoothed.fits"]
      
#FOV = [400,800]#pc
FOV = [70,360]#pc

Min_res=4.3*u.pc#Smallest resolution


iterations = 6
Line_Name = '_NGC_HCN_J1_0_'
ovs = 3 #over sample factor for beam
min_vel = 3.3#km/s
#Beam_Sizes = np.linspace(Smallest_beam,Smallest_beam*iterations, np.diff(Smallest_beam,Smallest_beam*iterations)/iterations)
iter_factor = 1/5
iterations = 6
for kl in range(5,6):
    Prime_Beam = Min_res*((kl*iter_factor+1))#Beam_Sizes[kl]
    if(kl==5):
        Prime_Beam = 4.3*u.pc#Nico
        ovs=5
        FOVp=[70,360]
    #Need to break it up into 30-wide vel slices to do the reprojection (ram-draw too high)
    
    for i in range(0,300):
        
        Cube_Name = str(Prime_Beam.value)+"pc_beam_"+str(i)+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc.fits'
        sc = SpectralCube.read(files[0])  

        if(i*30>len(sc)):
            print("End Loop at",i)
            break

        sc = sc[i*30:i*30+30,0:np.shape(sc)[1]-0,0:np.shape(sc)[2]-0]# Make a subcube
        #sc = sc[i*30:i*30+30,250:400,50:400]# Make a subcube
        #Put in the right system
        sc_kms = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
        del sc

        sc = sc_kms.spectral_slab(0. *u.km / u.s, 501. *u.km / u.s)  # Crop out velocities we don't care about    
        del sc_kms
        sc.allow_huge_operations=True    
        HI = sc.header
        Nres=Prime_Beam
        dist=3.5*10**6
        res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc
        beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)
        sc.allow_huge_operations=True
        
        
        
        gal="NGC253"



        #Now that we have a circular beam, this can be done easily:
        #use some trig to make a rectangle at 33 deg, and allow pixels inside it.
        if(gal == 'NGC253'):

            cdelt_x = u.Quantity(str(np.abs(sc.header['cdelt1']))+sc.header['cunit1'])
            cdelt_y = u.Quantity(str(np.abs(sc.header['cdelt2']))+sc.header['cunit2'])
            #center = SkyCoord('00h47m33.134s' ,'-25d17m19.68s',frame='icrs')
            center = SkyCoord('00h47m33.14s' ,'-25d17m17.52s',frame='icrs')

            center_ra_pix,center_dec_pix = [int(sc.wcs[:][:][0].world_to_pixel(center)[0]),int(sc.wcs[:][:][0].world_to_pixel(center)[1])]
            PixFov = [int(FOVp[0]/(cdelt_x/u.deg*np.pi/180*3.5*10**6))/2,int(FOVp[1]/(cdelt_y/u.deg*np.pi/180*3.5*10**6))/2]
            print(center_ra_pix,center_dec_pix)
            pixels = np.zeros(np.shape(sc))           
            for lmi in range(np.shape(sc)[0]):
                for lmj in range(np.shape(sc)[1]):
                    for lmk in range(np.shape(sc)[2]):
                        hypo = np.sqrt(((lmj-center_dec_pix)**2) + (lmk-center_ra_pix)**2)

                        if (lmj-center_dec_pix!=0):
                            ang = np.arctan(abs(lmk-center_ra_pix)/abs(lmj-center_dec_pix))#Find angle to the center
                        else:
                            ang = np.pi/2
                        if(lmk>center_ra_pix and lmj>center_dec_pix):
                            ang*=-1
                            #print("A",lmk,lmj,ang)
                        elif(lmk<center_ra_pix and lmj<center_dec_pix):
                            ang*=-1
                            #print("B",lmk,lmj,ang)
                        elif(lmk>center_ra_pix and lmj<center_dec_pix):
                            if ang >57*np.pi/180:
                                #print("C",lmk,lmj,ang)

                                ang= -33*np.pi/180+147*np.pi/180-ang#coming from the opposite end of the ra axis now, but projecting still to 33 deg from north.

                                #print(ang,hypo*np.cos(abs((33*np.pi/180)+ang)),hypo*np.sin(abs((33*np.pi/180)+ang)))
                            else:
                                #print(ang,hypo*np.cos(abs((33*np.pi/180)+ang)),hypo*np.sin(abs((33*np.pi/180)+ang)))
                                #print("D",lmk,lmj,ang)
                                pass


                        up_pixels = abs(hypo*np.cos(abs((33*np.pi/180)+ang)))#Should not be over the fov in the upwards direction (relative to 0 degrees)
                        side_pixels = abs(hypo*np.sin(abs((33*np.pi/180)+ang)))#Should not be over the fov in the side-side direction (relative to 0 degrees)

                        if(up_pixels<PixFov[0] and side_pixels<PixFov[1]):
                            pixels[lmi][lmj][lmk] = 1#good
                    #print(up_pixels,side_pixels ,PixFov)
                    #print(    center_ra_pix,    center_dec_pix,PixFov,side_pixels,up_pixels,ang,lmi,ggg,fff,pixels[lmi][ggg][fff])
            #print(pixels)
            bp = np.where(pixels!=1)
            #Mask teh pixels outside the fov
            scCopy = sc.hdu
            #sc.hdu.data[bp]=9999#np.nan
            scCopy.data[bp]=np.nan
            sc = SpectralCube.read(scCopy)
            del scCopy
            
            
        #Get right size
 
        sc.allow_huge_operations=True
        datn = sc.hdu.data
        sx,sy,ex,ey=0,0,0,0
        for lmi in range(np.shape(datn[0,:,:])[0]):

            if(ey!=0 and sx!=0 and ex!=0 and sy!=0):
                print("F",lmi)
                break
            for lmj in range(np.shape(datn[0,:,:])[1]):

                if(sx==0):            
                    if(np.nanmean(datn[0,lmi,:])>0 or np.nanmean(datn[0,lmi,:])<0):
                        sx=lmi


                if(sy==0):
                    if(np.nanmean(datn[0,:,lmj])>0 or np.nanmean(datn[0,:,lmj])<0):
                        sy=lmj

                if(ex==0):
                    if(np.nanmean(datn[0,np.shape(datn[0,:,:])[0]-lmi-1,:])>0 or np.nanmean(datn[0,np.shape(datn[0,:,:])[0]-lmi-1,:])<0):
                        ex=np.shape(datn[0,:,:])[0]-lmi-1

                if(ey==0):
                    if(np.nanmean(datn[0,:,np.shape(datn[0,:,:])[1]-lmj-1])>0 or np.nanmean(datn[0,:,np.shape(datn[0,:,:])[1]-lmj-1])<0):
                        ey=np.shape(datn[0,:,:])[1]-lmj-1

                if(ey!=0 and ex!=0 and sx!=0 and sy!=0):
                    break
        print(sx,ex,sy,ey)
        scP = sc[:,sx:ex,sy:ey]
        scP_Hdu=scP.hdu
        zeros=((scP_Hdu.data[:,:,:]==0))
        bp = np.where(zeros)
        scP_Hdu.data[bp]=np.nan
        sc = SpectralCube.read(scP_Hdu)

        sc = sc.convolve_to(beam)#Smoothe to circular beam at 3 pc by 3 pc
        del scP_Hdu
        del zeros
        del scP
        del datn
        



        
        
        

        reheader = sc.header



        ## Find the number of expected pixels for the new resolution and the location of the left/right, up/down sides 
        if sc.header['cdelt1']>0:
            pix_x    = (res/ovs).to(u.degree).value
            origin_x = sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value

        else:
            pix_x    = -1.*(res/ovs).to(u.degree).value
            origin_x = (sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value

        if sc.header['cdelt2']>0:
            pix_y    = (res/ovs).to(u.degree).value
            origin_y = sc.latitude_extrema[0].to(u.degree).value
        else:
            pix_y    = -1.*(res/ovs).to(u.degree).value
            origin_y = sc.latitude_extrema[1].to(u.degree).value

        if gal=='NGC253':
            npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)

        elif gal=='GC':
            npix_x   = int(np.ceil((sc.longitude_extrema[0]-(sc.longitude_extrema[1]-360*u.degree))/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)

        #Correct the header to the expected pixels for the new res

        reheader['cdelt1'] = pix_x
        reheader['cdelt2'] = pix_y

        reheader['naxis1'] = npix_x
        reheader['naxis2'] = npix_y

        reheader['crval1'] = origin_x
        reheader['crval2'] = origin_y

        reheader['crpix1'] = 0
        reheader['crpix2'] = 0
        try:
            del reheader['lonpole']
            del reheader['latpole']
            del reheader['wcsaxes']#Dont need these anymore, maybe?
        except:
            pass

        # regrid cube to target pixel size
        sc_K_kms = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
        
        new = SpectralCube(data=sc_K_kms.hdu.data,wcs = WCS(sc_K_kms.header),header=sc_K_kms.header)
        new.allow_huge_operations=True
        new = new*sc_K_kms[0][0][0].unit
        #do this because scs dont like being modified
        del sc_K_kms
        sc_K_kms = new
        del new

        print(np.nanmax(sc_K_kms),np.shape(sc_K_kms))#These should be a non zero float and the shape of the cube (30,~1000,~1000)
        del sc
        vel = np.arange(0,501,min_vel)*u.km/u.s
        NGC_CO_J3_2 = sc_K_kms
        del sc_K_kms
        NGC_CO_J3_2.allow_huge_operations=True

        #do this again to crop the extra pixels off
        sc=NGC_CO_J3_2
        datn = sc.hdu.data
        sx,sy,ex,ey=0,0,0,0
        for lmi in range(np.shape(datn[0,:,:])[0]):

            if(ey!=0 and sx!=0 and ex!=0 and sy!=0):
                print("F",lmi)
                break
            for lmj in range(np.shape(datn[0,:,:])[1]):

                if(sx==0):            
                    if(np.nanmean(datn[0,lmi,:])>0 or np.nanmean(datn[0,lmi,:])<0):
                        sx=lmi


                if(sy==0):
                    if(np.nanmean(datn[0,:,lmj])>0 or np.nanmean(datn[0,:,lmj])<0):
                        sy=lmj

                if(ex==0):
                    if(np.nanmean(datn[0,np.shape(datn[0,:,:])[0]-lmi-1,:])>0 or np.nanmean(datn[0,np.shape(datn[0,:,:])[0]-lmi-1,:])<0):
                        ex=np.shape(datn[0,:,:])[0]-lmi-1

                if(ey==0):
                    if(np.nanmean(datn[0,:,np.shape(datn[0,:,:])[1]-lmj-1])>0 or np.nanmean(datn[0,:,np.shape(datn[0,:,:])[1]-lmj-1])<0):
                        ey=np.shape(datn[0,:,:])[1]-lmj-1

                if(ey!=0 and ex!=0 and sx!=0 and sy!=0):
                    break
        print(sx,ex,sy,ey)
        scP = sc[:,sx:ex,sy:ey]
        scP_Hdu=scP.hdu
        zeros=((scP_Hdu.data[:,:,:]==0))
        bp = np.where(zeros)
        scP_Hdu.data[bp]=np.nan
        sc = SpectralCube.read(scP_Hdu)

        sc = sc.convolve_to(beam)#Smoothe to circular beam at 3 pc by 3 pc
        del scP_Hdu
        del zeros
        del scP
        del datn
        
        #Write the intermediary cubes that will then be spliced together
        sc.write(Cube_Name,overwrite=True)      
        del NGC_CO_J3_2
        del sc
        gc.collect()######################################################################




print("done")         

577 648
F 408
407 889 255 899
0.0 1.4368850473684927
2.2800652280150217e-09 1.0864614693810945e-10
0.002107551651757235 (22, 295, 435)
F 2
1 293 1 392
577 648
F 408
407 889 255 899
0.0 1.4368850473684927
2.2800652280150217e-09 1.0864614693810945e-10
0.009315924810366813 (30, 295, 435)
F 2
1 293 1 392
577 648
F 408
407 889 255 899
0.0 1.4368850473684927
2.2800652280150217e-09 1.0864614693810945e-10
0.026027169794102877 (30, 295, 435)
F 2
1 293 1 392
577 648
F 408
407 889 255 899
0.0 1.4368850473684927
2.2800652280150217e-09 1.0864614693810945e-10
0.012922757220390524 (30, 295, 435)
F 2
1 293 1 392
577 648
F 408
407 889 255 899
0.0 1.4368850473684927
2.2800652280150217e-09 1.0864614693810945e-10
0.0018386003467938258 (30, 295, 435)
F 2
1 293 1 392
577 648
F 408
407 889 255 899
0.0 1.4368850473684927
2.2800652280150217e-09 1.0864614693810945e-10
0.0007574618267489862 (11, 295, 435)
F 2
1 293 1 392


ValueError: attempt to get argmin of an empty sequence

In [3]:
#Splice hcn

iterations = 6
iter_factor = 1/5
Line_Name = '_NGC_HCN_J1_0_'
ovs = 3 #over sample factor for beam
min_vel = 3.3#km/s
FOV = [70,360]#pc
Min_res=4.3*u.pc#Smallest resolution

for kl in range(5,6):
      
    Prime_Beam = Min_res*((kl*iter_factor+1))#
    if(kl==5):
        Prime_Beam = 4.3*u.pc#Nico
        ovs=5
        FOVp=[70,360]
    print(Prime_Beam)
    
    for km in range(5,6):
        vel_prime = min_vel*((km*iter_factor+1))
        
        if(km==5):
            vel_prime=3.3
        vel = np.arange(0,501,vel_prime)*u.km/u.s 
        
        
        for i in range(6):
            
            Cube_Name_Load = str(Prime_Beam.value)+"pc_beam_"+str(i)+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc.fits'
            Cube_Name_Save = str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+'_vel_res.fits'
            
            sc=SpectralCube.read((Cube_Name_Load)) 
            print(np.shape(sc),Cube_Name_Load)
            print(sc.mask,type(sc.mask))

            if i==0:
                reheader = sc.header
                rewcs = sc.wcs


            print(type(sc))
            if i ==0:
                scW=SpectralCube.read((Cube_Name_Load))      
                mask = scW.mask.include() #Need to create a mask because it doesn't get splcied
            else:


                if i ==1:
                    scW = np.concatenate((scW[:].hdu.data,sc[:].hdu.data),dtype = type(sc))
                    print(type(scW))
                    mask = np.concatenate((mask[:],sc[:].mask.include()),dtype = type(sc[:].mask.include()))
                else:
                    scW = np.concatenate((scW[:],sc.hdu.data[:]),dtype = type(sc))
                    mask = np.concatenate((mask[:],sc[:].mask.include()),dtype = type(sc[:].mask.include()))


            print(np.shape(scW))

        def duh(lol):
            gp = np.where(lol!=np.nan)
            lol[gp]=True
            return lol #Anywhere that has data will be unmasked
        reheader["NAXIS3"] = len(scW)
        Full_Mask = LazyMask(function = duh,data = mask, wcs = rewcs)
        scWsc = SpectralCube(data = scW,wcs = rewcs, header = reheader, mask = Full_Mask)#The spliced cube

        scWsc.allow_huge_operations=True
        scWsc = scWsc*sc[0][0][0].unit#Add unit back in
        del sc
        fwhm_factor = np.sqrt(8*np.log(2))

        scWsc.allow_huge_operations=True


        #scWsc = scWsc.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))
        #scWsc = scWsc.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))
        vel = np.arange(0,501,vel_prime)*u.km/u.s    

        G_width = np.sqrt(abs(scWsc.header["CDELT3"]**2-vel_prime**2))
        if(km==5):
            G_width = np.sqrt(abs(3.3**2-1.8**2))
        scWsc = scWsc.spectral_smooth(Gaussian1DKernel(G_width/fwhm_factor))#Preserves information from the pixels lost in downsampling

        scWsc = scWsc.spectral_interpolate(spectral_grid=vel) # Match velocities to -250 251 range 



        scWsc.write(Cube_Name_Save,overwrite=True)
        #scWsc.write("NGC_Spliced_Reprojected_Whole_CO_32.fits",overwrite=True)


print("done")   

4.3 pc
(22, 292, 391) 4.3pc_beam_0_NGC_HCN_J1_0_70x360pc.fits
<spectral_cube.masks.LazyMask object at 0x7fc6a03ba040> <class 'spectral_cube.masks.LazyMask'>
<class 'spectral_cube.spectral_cube.SpectralCube'>
(22, 292, 391)
(30, 292, 391) 4.3pc_beam_1_NGC_HCN_J1_0_70x360pc.fits
<spectral_cube.masks.LazyMask object at 0x7fc5d678afd0> <class 'spectral_cube.masks.LazyMask'>
<class 'spectral_cube.spectral_cube.SpectralCube'>
<class 'numpy.ndarray'>
(52, 292, 391)
(30, 292, 391) 4.3pc_beam_2_NGC_HCN_J1_0_70x360pc.fits
<spectral_cube.masks.LazyMask object at 0x7fc6a0461ca0> <class 'spectral_cube.masks.LazyMask'>
<class 'spectral_cube.spectral_cube.SpectralCube'>
(82, 292, 391)
(30, 292, 391) 4.3pc_beam_3_NGC_HCN_J1_0_70x360pc.fits
<spectral_cube.masks.LazyMask object at 0x7fc5d8073970> <class 'spectral_cube.masks.LazyMask'>
<class 'spectral_cube.spectral_cube.SpectralCube'>
(112, 292, 391)
(30, 292, 391) 4.3pc_beam_4_NGC_HCN_J1_0_70x360pc.fits
<spectral_cube.masks.LazyMask object at 0x7fc5d97

In [7]:
#Fix reprojected repeated pixels

iterations = 6
iter_factor = 1/5
Line_Name = '_NGC_HCN_J1_0_'
ovs = 3 #over sample factor for beam
min_vel = 3.3#km/s
FOV = [70,360]#pc
Min_res=4.3*u.pc#Smallest resolution

for kl in range(5,6):
      
    Prime_Beam = Min_res*((kl*iter_factor+1))#
    if(kl==5):
        Prime_Beam = 4.3*u.pc#Nico
        ovs=5
        FOVp=[70,360]
    print(Prime_Beam)
    
    
    for km in range(5,6):
        vel_prime = min_vel*((km*iter_factor+1))
        if(km==5):
            vel_prime=3.3
        
        
        vel = np.arange(0,501,vel_prime)*u.km/u.s 
        
        Cube_Name_Load = str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+'_vel_res.fits'
    
    
        Cube_Name_Save = "Cropped_"+str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+'_vel_res.fits'
        


        sc = SpectralCube.read(Cube_Name_Load).with_spectral_unit(u.km/u.s,velocity_convention="radio")
        sp=0
        for lmi in range(len(sc)):
            #Check to see if the slice has been repeated by the interpolation function
            if(np.round(np.nanmean(sc[lmi].hdu.data),5)==np.round(np.nanmean(sc[lmi+1].hdu.data),5)):

                sp = lmi+1
            else:
                print("A")
                print(lmi,np.nanmean(sc[lmi].hdu.data),np.nanmean(sc[lmi+1].hdu.data))
                break
        l = len(sc)-1
        ep=l
        for lmi in range(l):

            if(np.round(np.nanmean(sc[l-lmi].hdu.data),5)==np.round(np.nanmean(sc[l-lmi-1].hdu.data),5)):
                ep = l-lmi-1

            else:
                print("B",lmi)
                break

        print(sp,ep)#These are the start and stop slices where the actual unique data resides

        sc.allow_huge_operations=True
        sc = sc[sp:ep]
        
        
        
        scP_Hdu=sc.hdu
        zeros=((scP_Hdu.data[:,:,:]==0))
        bp = np.where(zeros)
        scP_Hdu.data[bp]=np.nan
        sc = SpectralCube.read(scP_Hdu)
        
        
        
        sc = sc.to(u.K)
        
        sc.write((Cube_Name_Save),overwrite=True)
        del sc
        del scP_Hdu
        
        print("Done",km,kl)

4.3 pc
A
13 3.851897586990725e-06 5.918066287419639e-06
B 24
13 127
Done 5 5
